## 数据预处理
创建一个人工数据集，并存储在 csv（逗号分隔值）文件

In [1]:
# 数据预处理的意思是如果我有一个原始数据，我怎么将他读取进来并使用机器学习的方法进行处理
import os

os.makedirs(os.path.join('.', 'data'), exist_ok=True)
data_file = os.path.join('.', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行表示一个数据名
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')
    # NA 表示未知

### 从创建的 csv 文件中加载原始数据集
一般读取 csv 文件使用 python 中 pandas 这个库

In [2]:
#如果没有安装 pandas，在Jupyter Notebook 中取消下面注释就可以安装了
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [3]:
# 在 jupyter 直接 data 可能更美观
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [8]:
# 为了处理缺失的数据，典型的方法包括 插值 和 删除，这里，我们将考虑插值
# “机器学习就是处理缺失的数据”
# iloc = index location
# 我们将 data 这个 4 x 3 的表使用 iloc 将第零列和第一列的所有的行放在 inputs 里
# 我们将 data 这个 4 x 3 的表使用 iloc 将第 2 列（最后一列）拿出来放在 outputs 里
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean(numeric_only=None)) # 在 inputs 里面使用 fillna 将所有 NAN(Not A Number) 填成剩下所有不是 NAN 的数值的均值，mean 是均值的意思
# 比如说 我们的 NumRooms 这一列，有两个 NAN，填成 剩下 的 2+4/2 = 3 
# Alley 这一列是个 string，所以没有均值可言
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


C:\Users\Administrator\AppData\Local\Temp\ipykernel_22900\3420005972.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean(numeric_only=None)) # 在 inputs 里面使用 fillna 将所有 NAN(Not A Number) 填成剩下所有不是 NAN 的数值的均值，mean 是均值的意思


In [5]:
# 对于上面这种不是数值（比如说 string 类型的）但是需要插值的怎么办？
# 对于 input 中的类别值或离散值，我们将 `NAN` 视为一个类别
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)
# 个人理解：说白了，就是将一些不能使用 inputs.mean() 的数值采用分类方法进行插值

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


In [6]:
# 现在 inputs 和 outputs 中的所有条目都是数值类型，他们可以转换为张量格式

import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## Q & A

### Q1

In [7]:
a = torch.arange(12)
b = a.reshape((3,4))
b[:] = 2
a
# 可以理解为 reshape 是原地操作，无返回，故 b 指向 a；view 返回新的 Tensor 对象，故 b 指向新对象

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])